In [117]:
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline

##make sure that vector x is inside a box
##of length L (centred on the origin)
def imageVec(x, L):
    hL = 0.5 * L
    for d in range(2):
        while x[d] >  hL:
            x[d]-=L
        while x[d] < -hL:
            x[d]+=L
    return

##draw circles of radius one at positions x
##with vectors f at the centres.
def plotCircles(x, f, L):
    circList  = []
    arrowList = []
    for i in range(len(x)):
        circle = plt.Circle((x[i,0], x[i,1]), 0.5, color='limegreen', alpha = 0.5)
        circList.append(circle)
        
        ##show image particles also
        corner = False
        if x[i,0] > L/2 - 0.5:
            if x[i,1] > L/2 - 0.5:
                circList.append( plt.Circle((x[i,0]-L, x[i,1]-L), 0.5, color='b', alpha = 0.2) )
            if x[i,1] < -L/2 + 0.5:
                circList.append( plt.Circle((x[i,0]-L, x[i,1]+L), 0.5, color='b', alpha = 0.2) )
            circList.append( plt.Circle((x[i,0]-L, x[i,1]), 0.5, color='b', alpha = 0.2) )
                
        if x[i,0] < -L/2 + 0.5:
            if x[i,1] > L/2 - 0.5:
                circList.append( plt.Circle((x[i,0]+L, x[i,1]-L), 0.5, color='b', alpha = 0.2) )
            if x[i,1] < -L/2 + 0.5:
                circList.append( plt.Circle((x[i,0]+L, x[i,1]+L), 0.5, color='b', alpha = 0.2) )
            circList.append( plt.Circle((x[i,0]+L, x[i,1]), 0.5, color='b', alpha = 0.2) )
                
        if x[i,1] > L/2 - 0.5:
            circList.append( plt.Circle((x[i,0], x[i,1]-L), 0.5, color='b', alpha = 0.2) )
        if x[i,1] < -L/2 + 0.5:
            circList.append( plt.Circle((x[i,0], x[i,1]+L), 0.5, color='b', alpha = 0.2) )
        
        arrow  = plt.arrow( x[i,0], x[i,1], 0.05*f[i,0], 0.05*f[i,1],\
                           color='b', head_width=0.1 )
        arrowList.append(arrow)
        
    ##get an axis and clear it.
    ax = plt.gca()
    ax.cla() 
    
    ##place the circles and arrows on the axis:
    for circ in circList:
        ax.add_artist(circ)
    for arrow in arrowList:
        ax.add_artist(arrow)
        
    ##set size and show
    ax.set_xlim((-L/2, L/2))
    ax.set_ylim((-L/2, L/2))
    ax.set_aspect('equal')
    plt.title('disks in a box')
    plt.show()

####### define a random force vector, as a test for the plotting function
frc      = np.random.random((N,2)) - 0.5



##  !!!!! MY CODE IS BELOW !!!!!



## Final Comments
print("I know that my code is partially, if not entirely wrong. The issue is that I am 95% certain that I don't understand the cost formula correctly. I don't see how to obtain a 2D gradient if I take the gradient of a scalar function, unless it is itself 2D, which then means my calculations of the cost is wrong. This is strongly related to the fact that I have used the same gradient as for 1D, and didn't try to do it for 2D. I have nevertheless tried to finish what was asked, as if the not-so-working code was actually doing what I wanted")


## --- initialization --- 
print("===== initialization =====")
L = 2.7                ## size of the box
N = 5                  ## amount of discs in the box
D = 1.                 ## diameter of the discs
C = np.zeros((N,N))    ## inverse-square repulsive force
newC = np.zeros((N,N)) ## new cost, useful for later comparison
R = np.zeros((N,N))    ## distances between centers of discs
S = np.zeros((N,2))    ## random system state vector
G = 1e-5               ## smol gamma (no epsilon? I've been lied to for so long...)
iterationNumber = 10   ## amount of times we do the optimizing process
iterationAmount = 0    ## to track the actual needed iterations
absValueJ = 0          ## absolute value of our J gradient vector
cVersusIterations = []

## --- debug ---
print("----- Debug time -----")
print("L = %.1f" %L)
print("N = %i" %N)
print("D = %.1f" %D, "\n")
print("C = ", C, "\n")
print("R = ", R, "\n")
print("S = ", S, "\n")
print("G = %.0e" %G)
print("----- Debug end -----")
print("\n")

## reusing code from assignment 7
class Differentiator():
    """
    This class contains our gradient operator 
    """
    def __init__( self, N_in, L_in):
        
        self.N = N_in
        self.L = L_in
        self.gradientOperator = np.zeros((N,N))
        dx = self.L / self.N
        
        ## defining the gradient
        for i in range(N):
            self.gradientOperator[i,(i+1)%N] =  1.0
            self.gradientOperator[i,(i-1)%N] = -1.0
        self.gradientOperator /= (2*dx)
        
    ## the function applying the gradient operator
    def gradient(self, y):
        gradient = self.gradientOperator @ y
        
        return gradient

## --- int main(void) --- 
print("===== main =====")
def costCalculator(S):
    ## getting the distances matrix
    print("// getting the distances matrix")
    for i in range(N):
        for j in range(N):
            R[i][j] = round(np.sqrt((S[i][0] - S[j][0])**2 + (S[i][1] - S[j][1])**2),2) ## round for readability, remove later
        
    ## --- debug ---
    print("----- Debug time -----")
    print("R = ", R)
    print("----- Debug end -----")
    print("\n")

    ## calculating the cost
    print("// calculating the cost")
    for i in range(N):
        for j in range(N):
            if i != j and np.abs(R[i][j]) < 1 and R[i][j]!=0:
                C[i][j] += 1/R[i][j] - 1
                C[i][j] /= 2
                
    
    return C

## generating the random system vector (random starting condition)
print("// generation of the random system vector")
for i in range(N):
    for j in range(2):
        S[i][j] = L * np.random.random() - 0.5 * L ## a disc with center inside the box

## --- debug ---
print("----- Debug time -----")
print("S = ", S)
print("----- Debug end -----")
print("\n")




"""
    see cell below for uncommented version
"""




# for k in range(iterationNumber): ## if this were not commented, need to indent all not commented code
#     iterationAmount += 1
    
C = costCalculator(S)
## --- debug ---
print("----- Debug time -----")
print("C = ", C)
print("----- Debug end -----")
print("\n")
    
## getting the gradient of the cost
print("// getting the gradient of the cost")
scalarC = []
for i in range(N):
    scalarC.append(0)

for i in range(len(scalarC)):
     for j in range(N):
        scalarC[i] += C[j][i]

diff = Differentiator(N,L) ## calling the instance of the class

J = diff.gradient(scalarC)

for i in range(len(J)):
    absValueJ += J[i]**2

# --- this bit doesn't work because my gradient isn't 2D
#     if absValueJ < G:
#         break
#     else:
#         T = S - G * J / absValueJ
#         S,T = T,S ## I do read the feedback!
    
## --- debug ---
print("----- Debug time -----")
print("scalarC = ", scalarC)
print("J = ", J)
print("absValueJ = ", absValueJ)
print("----- Debug end -----")
print("\n")
    
# plotCircles(S ,frc, L)
    
## repeat
# newC,C = C,newC
#     newC = copy.deepcopy(costCalculator(S))
    
# if newC > C:
#     print("break") ## if loop isn't commented out, this would just be a normal break
    
#    cVersusIterations.append(C)

# plt.scatter(iterationAmount, cVersusIterations)

I know that my code is partially, if not entirely wrong. The issue is that I am 95% certain that I don't understand the cost formula correctly. I don't see how to obtain a 2D gradient if I take the gradient of a scalar function, unless it is itself 2D, which then means my calculations of the cost is wrong. This is strongly related to the fact that I have used the same gradient as for 1D, and didn't try to do it for 2D. I have nevertheless tried to finish what was asked, as if the not-so-working code was actually doing what I wanted
===== initialization =====
----- Debug time -----
L = 2.7
N = 5
D = 1.0 

C =  [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]] 

R =  [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]] 

S =  [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]] 

G = 1e-05
----- Debug end -----


===== main =====
// generation of the random system vector
----- Debug time -----
S =  [[-0.14077379 -1.0

In [116]:
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline

##make sure that vector x is inside a box
##of length L (centred on the origin)
def imageVec(x, L):
    hL = 0.5 * L
    for d in range(2):
        while x[d] >  hL:
            x[d]-=L
        while x[d] < -hL:
            x[d]+=L
    return

##draw circles of radius one at positions x
##with vectors f at the centres.
def plotCircles(x, f, L):
    circList  = []
    arrowList = []
    for i in range(len(x)):
        circle = plt.Circle((x[i,0], x[i,1]), 0.5, color='limegreen', alpha = 0.5)
        circList.append(circle)
        
        ##show image particles also
        corner = False
        if x[i,0] > L/2 - 0.5:
            if x[i,1] > L/2 - 0.5:
                circList.append( plt.Circle((x[i,0]-L, x[i,1]-L), 0.5, color='b', alpha = 0.2) )
            if x[i,1] < -L/2 + 0.5:
                circList.append( plt.Circle((x[i,0]-L, x[i,1]+L), 0.5, color='b', alpha = 0.2) )
            circList.append( plt.Circle((x[i,0]-L, x[i,1]), 0.5, color='b', alpha = 0.2) )
                
        if x[i,0] < -L/2 + 0.5:
            if x[i,1] > L/2 - 0.5:
                circList.append( plt.Circle((x[i,0]+L, x[i,1]-L), 0.5, color='b', alpha = 0.2) )
            if x[i,1] < -L/2 + 0.5:
                circList.append( plt.Circle((x[i,0]+L, x[i,1]+L), 0.5, color='b', alpha = 0.2) )
            circList.append( plt.Circle((x[i,0]+L, x[i,1]), 0.5, color='b', alpha = 0.2) )
                
        if x[i,1] > L/2 - 0.5:
            circList.append( plt.Circle((x[i,0], x[i,1]-L), 0.5, color='b', alpha = 0.2) )
        if x[i,1] < -L/2 + 0.5:
            circList.append( plt.Circle((x[i,0], x[i,1]+L), 0.5, color='b', alpha = 0.2) )
        
        arrow  = plt.arrow( x[i,0], x[i,1], 0.05*f[i,0], 0.05*f[i,1],\
                           color='b', head_width=0.1 )
        arrowList.append(arrow)
        
    ##get an axis and clear it.
    ax = plt.gca()
    ax.cla() 
    
    ##place the circles and arrows on the axis:
    for circ in circList:
        ax.add_artist(circ)
    for arrow in arrowList:
        ax.add_artist(arrow)
        
    ##set size and show
    ax.set_xlim((-L/2, L/2))
    ax.set_ylim((-L/2, L/2))
    ax.set_aspect('equal')
    plt.title('disks in a box')
    plt.show()

####### define a random force vector, as a test for the plotting function
frc      = np.random.random((N,2)) - 0.5



##  !!!!! MY CODE IS BELOW !!!!!



## Final Comments
print("I know that my code is partially, if not entirely wrong. The issue is that I am 95% certain that I don't understand the cost formula correctly. I don't see how to obtain a 2D gradient if I take the gradient of a scalar function, unless it is itself 2D, which then means my calculations of the cost is wrong. I have nevertheless tried to finish what was asked, as if the not-so-working code was actually doing what I wanted")


## --- initialization --- 
print("===== initialization =====")
L = 2.7                ## size of the box
N = 5                  ## amount of discs in the box
D = 1.                 ## diameter of the discs
C = np.zeros((N,N))    ## inverse-square repulsive force
newC = np.zeros((N,N)) ## new cost, useful for later comparison
R = np.zeros((N,N))    ## distances between centers of discs
S = np.zeros((N,2))    ## random system state vector
G = 1e-5               ## smol gamma (no epsilon? I've been lied to for so long...)
iterationNumber = 10   ## amount of times we do the optimizing process
iterationAmount = 0    ## to track the actual needed iterations
absValueJ = 0          ## absolute value of our J gradient vector
cVersusIterations = []

## --- debug ---
print("----- Debug time -----")
print("L = %.1f" %L)
print("N = %i" %N)
print("D = %.1f" %D, "\n")
print("C = ", C, "\n")
print("R = ", R, "\n")
print("S = ", S, "\n")
print("G = %.0e" %G)
print("----- Debug end -----")
print("\n")

## reusing code from assignment 7
class Differentiator():
    """
    This class contains our gradient operator 
    """
    def __init__( self, N_in, L_in):
        
        self.N = N_in
        self.L = L_in
        self.gradientOperator = np.zeros((N,N))
        dx = self.L / self.N
        
        ## defining the gradient
        for i in range(N):
            self.gradientOperator[i,(i+1)%N] =  1.0
            self.gradientOperator[i,(i-1)%N] = -1.0
        self.gradientOperator /= (2*dx)
        
    ## the function applying the gradient operator
    def gradient(self, y):
        gradient = self.gradientOperator @ y
        
        return gradient

## --- int main(void) --- 
print("===== main =====")
def costCalculator(S):
    ## getting the distances matrix
    print("// getting the distances matrix")
    for i in range(N):
        for j in range(N):
            R[i][j] = round(np.sqrt((S[i][0] - S[j][0])**2 + (S[i][1] - S[j][1])**2),2) ## round for readability, remove later
        
    ## --- debug ---
    print("----- Debug time -----")
    print("R = ", R)
    print("----- Debug end -----")
    print("\n")

    ## calculating the cost
    print("// calculating the cost")
    for i in range(N):
        for j in range(N):
            if i != j and np.abs(R[i][j]) < 1 and R[i][j]!=0:
                C[i][j] += 1/R[i][j] - 1
                C[i][j] /= 2
                
    
    return C

## generating the random system vector (random starting condition)
print("// generation of the random system vector")
for i in range(N):
    for j in range(2):
        S[i][j] = L * np.random.random() - 0.5 * L ## a disc with center inside the box

## --- debug ---
print("----- Debug time -----")
print("S = ", S)
print("----- Debug end -----")
print("\n")

for k in range(iterationNumber): ## if this were not commented, need to indent all not commented code
    iterationAmount += 1
    
    C = costCalculator(S)
    ## --- debug ---
    print("----- Debug time -----")
    print("C = ", C)
    print("----- Debug end -----")
    print("\n")
    
    ## getting the gradient of the cost
    print("// getting the gradient of the cost")
    scalarC = []
    for i in range(N):
        scalarC.append(0)

    for i in range(len(scalarC)):
         for j in range(N):
            scalarC[i] += C[j][i]

    diff = Differentiator(N,L) ## calling the instance of the class

    J = diff.gradient(scalarC)

    for i in range(len(J)):
        absValueJ += J[i]**2

    #--- this bit doesn't work because my gradient isn't 2D
        if absValueJ < G:
            break
        else:
            T = S - G * J / absValueJ ## J and S need to be same dimension, but they are not !
            S,T = T,S ## I do read the feedback!
    
    ## --- debug ---
    print("----- Debug time -----")
    print("scalarC = ", scalarC)
    print("J = ", J)
    print("absValueJ = ", absValueJ)
    print("----- Debug end -----")
    print("\n")
    
    plotCircles(S ,frc, L)
    
    # repeat
    newC,C = C,newC
    newC = copy.deepcopy(costCalculator(S))
    
    if newC > C:
        print("break") ## if loop isn't commented out, this would just be a normal break
    
    cVersusIterations.append(C)

plt.scatter(iterationAmount, cVersusIterations)

I know that my code is partially, if not entirely wrong. The issue is that I am 95% certain that I don't understand the cost formula correctly. I don't see how to obtain a 2D gradient if I take the gradient of a scalar function, unless it is itself 2D, which then means my calculations of the cost is wrong. I have nevertheless tried to finish what was asked, as if the not-so-working code was actually doing what I wanted
===== initialization =====
----- Debug time -----
L = 2.7
N = 5
D = 1.0 

C =  [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]] 

R =  [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]] 

S =  [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]] 

G = 1e-05
----- Debug end -----


===== main =====
// generation of the random system vector
----- Debug time -----
S =  [[ 0.51680659 -0.69821891]
 [ 1.13092542 -0.37785209]
 [ 0.61849728  0.40994178]
 [-0.14124283 -0.73967947]
 [ 0.57078512 -0.03887121

ValueError: operands could not be broadcast together with shapes (5,2) (5,) 